In [84]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder
from sklearn import linear_model
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier



In [2]:
df1 = pd.read_csv("D:\Learning\Machine-Learning-\data\csgo.csv")
df1.head()

,map,day,month,year,date,wait_time_s,match_time_s,team_a_rounds,team_b_rounds,ping,kills,assists,deaths,mvps,hs_percent,points,result
0,Mirage,3.0,8.0,2018.0,3/8/2018,327.0,2906.0,16.0,13.0,215.0,17.0,2.0,21.0,2.0,5.0,45.0,Win
1,Mirage,2.0,8.0,2018.0,2/8/2018,336.0,2592.0,16.0,11.0,199.0,13.0,4.0,24.0,2.0,0.0,40.0,Lost
2,Mirage,31.0,7.0,2018.0,31/7/2018,414.0,2731.0,16.0,14.0,85.0,15.0,3.0,18.0,3.0,26.0,37.0,Win
3,Mirage,31.0,7.0,2018.0,31/7/2018,317.0,2379.0,11.0,16.0,93.0,12.0,2.0,15.0,2.0,16.0,30.0,Lost
4,Mirage,30.0,7.0,2018.0,30/7/2018,340.0,3467.0,15.0,15.0,94.0,33.0,5.0,20.0,5.0,30.0,83.0,Tie


In [65]:
x = df1.drop(columns=['day','month','year','date','team_a_rounds','team_b_rounds','result'],axis=1)
y = df1['result']

In [40]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2025)

In [66]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Impute missing values
    ('scaler', StandardScaler())                   # StandardScaler for normalization
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encoding
])

In [89]:
preprocessor = ColumnTransformer(transformers=[
    ("num_transformers",numeric_transformer,['wait_time_s','match_time_s','ping','kills','deaths','mvps','hs_percent','points']),
    ("cate_transformer",categorical_transformer,['map'])
])

In [90]:
model = Pipeline(steps=[
    ("preprocessor",preprocessor),
    ("classifier",RandomForestClassifier())
])

In [91]:
# Fit the model
model.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_transformers',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['wait_time_s',
                                                   'match_time_s', 'ping',
                                                   'kills', 'deaths', 'mvps',
                                                   'hs_percent', 'points']),
                                                 ('cate_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['map'])])),
                ('classifier', RandomForestClassifier())])

In [97]:
y_pred = model.predict(x_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

        Lost       0.68      0.81      0.74       105
         Tie       0.43      0.14      0.21        21
         Win       0.81      0.76      0.79       101

    accuracy                           0.73       227
   macro avg       0.64      0.57      0.58       227
weighted avg       0.71      0.73      0.71       227

